In [1]:
import os
from PIL import Image
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# Split set
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

# Path to image folders, 
data_path = fr'../Datasets/full/Training'
# os.listdir(data_path)

def remove_white_background(pixels):
    for pixel in pixels:
        if ((256 > pixel > 200)):
            pixel = 0

    return pixels

def get_rgb_pixels_onehut_labels(src):
    print("Starting...")
    newPixels = []
    y = np.empty(shape=[0, 1])

    for subdir in os.listdir(src):
        current_path = os.path.join(src, subdir)
        for file in os.listdir(current_path):
            img = Image.open(os.path.join(current_path, file))
            grey_img = img.convert('L')
            imgResize = grey_img.resize((24,24))
            pixels = list(imgResize.getdata())
            pixels = remove_white_background(pixels)
            newPixels.append(pixels)
            y = np.append(y, subdir)
    return newPixels, LabelBinarizer().fit_transform(y) # OneHot encode y

def process_files(src):
    X_grey_train = []
    X_grey_validation = []
    X_grey_test = []
    all_pixels, y = get_rgb_pixels_onehut_labels(src)

    X_train, X_validation, y_train, y_validation = train_test_split(all_pixels, y, test_size=0.2, random_state=1)
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=1)
    
    for pixels in X_train:       
        X_grey_train.append(pixels.copy())
        
    for pixels in X_validation:       
        X_grey_validation.append(pixels.copy())
        
    for pixels in X_test:       
        X_grey_test.append(pixels.copy())
    
    print("Finished")
    return np.asarray(X_grey_train), np.asarray(X_grey_validation), np.asarray(X_grey_test), y_train, y_validation, y_test

In [2]:
# Call process_files and assign variables
X_grey_train, X_grey_validation, X_grey_test, y_train, y_validation, y_test = process_files(data_path)

Starting...
Finished


In [3]:
print(X_grey_train)

[[254 254 254 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]


In [4]:
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix

# Call process_files and assign variables
X_grey_train, X_grey_validation, X_grey_test, y_train, y_validation, y_test = process_files(data_path)
# Fit/train train-datasets and store prediction vectors in variables

# Grey
mlpc_grey =  MLPClassifier(hidden_layer_sizes=(576), activation='relu', solver='adam')
mlpc_grey.fit(X_grey_train, y_train)
grey_val_result = mlpc_grey.predict(X_grey_validation)

print("Grey model")
print(classification_report(y_validation, grey_val_result))

Starting...
Finished
Grey model
              precision    recall  f1-score   support

           0       1.00      0.97      0.98       133
           1       0.96      1.00      0.98       110
           2       0.78      0.70      0.74       130
           3       1.00      0.84      0.91       134
           4       0.98      0.98      0.98       127
           5       0.99      0.97      0.98       140
           6       0.97      0.97      0.97       125
           7       0.99      0.98      0.99       127
           8       1.00      0.79      0.88       133
           9       1.00      0.99      1.00       120
          10       1.00      0.91      0.95       126
          11       0.96      0.97      0.96        99
          12       0.97      0.97      0.97       190
          13       1.00      0.44      0.62       135
          14       0.97      1.00      0.99       106
          15       1.00      0.99      1.00       129
          16       0.93      0.18      0.30      

c:\users\olive\documents\6. semester\applied artificial intelligence\ai_exam\.venv\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
